In [1]:
import urllib.request
import pandas as pd

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x29ae4621b38>)

In [3]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

C:\Users\bjkim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
C:\Users\bjkim\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [4]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [6]:
print('훈련 데이터 샘플의 개수 : {}'.format(len(train_df)))
print('테스트 데이터 샘플의 개수 : {}'.format(len(test_df)))

훈련 데이터 샘플의 개수 : 150000
테스트 데이터 샘플의 개수 : 50000


In [10]:
!pip install eunjeon
from torchtext import data # torchtext.data 임포트
from eunjeon import Mecab

In [11]:
# Mecab을 토크나이저로 사용
tokenizer = Mecab()

In [12]:
# 필드 정의
ID = data.Field(sequential = False,
                use_vocab = False) # 실제 사용은 하지 않을 예정

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs, # 토크나이저로는 Mecab 사용.
                  lower=True,
                  batch_first=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)

In [13]:
from torchtext.data import TabularDataset

In [14]:
train_data, test_data = TabularDataset.splits(
        path='.', train='ratings_train.txt', test='ratings_test.txt', format='tsv',
        fields=[('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True)

In [15]:
print(vars(train_data[0]))

{'id': '9976970', 'text': ['아', '더빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], 'label': '0'}


In [16]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [17]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


In [18]:
print(TEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x0000029A9B7F95C0>>, {'<unk>': 0, '<pad>': 1, '.': 2, '이': 3, '는': 4, '영화': 5, '다': 6, '고': 7, '하': 8, '도': 9, '의': 10, '가': 11, '은': 12, '에': 13, '을': 14, '보': 15, '한': 16, '..': 17, '게': 18, ',': 19, '들': 20, '!': 21, '지': 22, '를': 23, '있': 24, '없': 25, '?': 26, '좋': 27, '나': 28, '었': 29, '만': 30, '는데': 31, '너무': 32, '봤': 33, '적': 34, '안': 35, '로': 36, '정말': 37, '음': 38, '으로': 39, '것': 40, '아': 41, '네요': 42, '재밌': 43, '어': 44, '점': 45, '같': 46, '지만': 47, '진짜': 48, '했': 49, '에서': 50, '기': 51, '네': 52, '않': 53, '거': 54, '았': 55, '수': 56, '되': 57, '면': 58, 'ㅋㅋ': 59, '과': 60, '말': 61, '연기': 62, '인': 63, '주': 64, '잘': 65, '최고': 66, '~': 67, '내': 68, '평점': 69, '이런': 70, '어요': 71, '던': 72, '와': 73, '생각': 74, '할': 75, '왜': 76, '해': 77, '1': 78, '겠': 79, '스토리': 80, '습니다': 81, 'ㅋㅋㅋ': 82, '...': 83, '드라마': 84, '아니': 85, '그': 86, '싶': 87, '사람': 88, '듯': 89, '더': 90, '함': 91, '감동': 92, '때': 93, '배우': 94, '본': 

In [19]:
from torchtext.data import Iterator

In [20]:
batch_size = 5

train_loader = Iterator(dataset=train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)

In [21]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 30000
테스트 데이터의 미니 배치 수 : 10000


In [22]:
batch = next(iter(train_loader)) # 첫번째 미니배치
print(batch.text)

tensor([[ 412,  450, 1927,  703,   14,   94,   22,   26,   26,  184,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [ 270,    9,   11,  354,   16,    5,    6,  684,  200,   10,  240,   14,
          745, 1766,   50,  382, 2028,   12,  170,   14],
        [1078,   94,   20,   10, 8853, 1499,    3,   32, 6728,   25,   29,   38,
          435, 1477, 2056,    8,   58,  209,   55,   38],
        [ 861,   99,   93,  457,  448,   31,    0,   14,  218, 3528, 1656, 3905,
          218,    6, 1508,    4,   54,   32,  988,   18],
        [   0,    0,  129,  753,   15,  225,   49,   31,    2,   17,  142,   98,
            2,   17,    1,    1,    1,    1,    1,    1]])
